In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 한달살기, 한달살이 - 블로그 데이터 확인


In [21]:
def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(days = 1)
    elif '일' in s:
        date = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        date = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        date = now - timedelta(weeks = int(s[:-3]))

    return date.strftime('%Y.%m.%d')

In [65]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query='

keyword_list = ['제주도', '한달살기']
keyword_search = ''

for word in keyword_list:    
    keyword_search += f'+{quote(word)}'

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산', '이벤트', '리브애니웨어', '미스터멘션', '탁송', '퇴실청소']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + keyword_search + delete_search
driver = webdriver.Chrome()
driver.get(url)

round(random.uniform(1.5, 2.5), 3)

while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

round(random.uniform(4.5, 6.5), 3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.lst_view').select('.view_wrap')

result = []

for li in lis[190:200]:
    try :
        title = li.select_one('.title_area').select_one('a').text.strip()
        #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
    except:
        title = 'error'
    
    try :
        content = li.select_one('.dsc_link').text.strip()
        #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
    except :
        content = 'error'
    try :          
        blog_link = li.select_one('.title_area').select_one('a')['href']        
    except :
        blog_link = 'error'

    try :          
        naver_time = li.select_one('.sub').text.strip()
        blog_time = convert_date(naver_time)
    except :
        blog_time = 'error'

    if blog_link.find('blog.naver.com') > 0:
                            
        driver = webdriver.Chrome()
        driver.get(blog_link)
        driver.switch_to.frame('mainFrame')
        time.sleep(round(random.uniform(0.0, 1.2), 3))

        try : 
            lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
        except : 
            try :
                lis = driver.find_element(By.ID, 'postListBody').find_elements(By.CLASS_NAME, 'se_textarea')
            except :
                pass

        
        
        blog_contents = []     

        for li in lis:
            #sub_content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())

            try : 
                sub_content = li.text.strip()
                blog_contents.append(sub_content.replace('\U0001f979','').replace('\n',''))
                blog_text = ' '.join(blog_contents)
            except :
                blog_text = 'error'

        time.sleep(round(random.uniform(0.5, 1.5), 3))

   
    result.append({'title' : title,
                'contents' : content,
                'time' : blog_time,
                'blog_text' : blog_text})


In [69]:
result[result['blog_text'] == 'error']

,title,contents,time,blog_text


In [70]:
result = pd.DataFrame(result)
result

,title,contents,time,blog_text
0,"제주도 한달살기 #55. 별방진, 하도해변","제주도 한달살기_ [DAY 22] 2022.4.25 55. 별방진, 하도해변 세화에...",2022.12.27,"제주도 한달살기_[DAY 22]2022.4.25 55. 별방진, 하도해변세화에서 시..."
1,함덕해수욕장 숙소 제주도 한달살기 하고 싶은 신흥스테이,제주 바다는 말해모해ㅠ.ㅠ 사진 보니까 또 가고싶네... 후.. 당분간은 사진으로 ...,2023.01.22,안녕하세요~! :)이번에 앨범 정리를 하면서 포스팅 안했던 제주도 사진이너무 많은거...
2,"제주, 제주도 한달살기 숙소 _ 연동",한달살기 숙소를 사진만 보고 예약을 하시는 것이기에 자세하게 찍어서 올려 드려요! ...,2017.05.01,"제주, 제주도 한달살기 숙소 _ 연동 제주시 연동, 한달살기 숙소오픈이 되었습니다...."
3,"뚜벅이 제주도한달살기 4일차 :) 협재_ 베롱정원, 메리아일랜드...",매일하지 않지만 그래도 나름 꾸준하게 저의 공간으로 가꾸어왔었거든요 :) 지난 제주...,2023.08.30,📍주소 : 제주 제주시 한림읍 협재4길 19📍영업시간 :10 :00 - 18:00목...
4,제주도 한달살기 숙소 추천 제주 조천읍 한달살이 스테이소소,제주도 한달살기 숙소 추천 ✔️ 제주 조천읍 한달살이 숙소 ✔️ 스테이소소 스테이소...,2023.03.28,제주도 한달살기 숙소 추천 ✔️제주 조천읍 한달살이 숙소 ✔️스테이소소 친구가 운...
5,여자혼자 제주도 한달살기하며 지냈던 괜찮은 에어비앤비 숙소,댓글 달아주시면 호스트님 번호공유 ☑️ 문의시 블로거 조은아 소개로 연락했다고 하셔...,2020.10.22,2019.12~2020.1월까지 40일동안 지냈던 제주도숙소.한달동안 지냈던 기록...
6,제주도 2주살기 한달까지 해결되는 함덕 시골집민박,오랫만에 제주도 한달살기 2주살기 일주일 살기 이야기로 찾아왔습니다. 제주시 동쪽에...,2020.05.20,오랫만에 제주도 한달살기 2주살기 일주일살기 이야기로 찾아왔습니다. 제주시 동쪽에 ...
7,[제주도] 제주한달살기숙소 찾으신다면 조용한 마을의...,안녕하세요 ❣️ 오늘은 제주도 한달살기숙소 포스팅 해볼게요 제주도 서귀포 표선면에 ...,2023.12.27,안녕하세요 ❣️오늘은 제주도 한달살기숙소 포스팅 해볼게요제주도 서귀포 표선면에 있는...
8,제주도 한달살기 숙소 제주 올레길 한달살이 숙소 추천,"알파캠프(걸어서 제주도 한바퀴)***.****.**** 제주도 한달살기 숙소,제주도...",2022.01.08,제주도 한달살기 숙소 제주 올레길 한달살이 숙소 추천 제주도 한달살기 숙소 제공은 ...
9,"[제주도 한달살기] Day 29 - 돈내코, 삼성혈해물탕, 하라케케...",사실상 제주도에서의 마지막 날. 다음날 저녁 비행기로 서울로 떠난다. 오늘은 그동안...,2023.11.10,사실상 제주도에서의 마지막 날.다음날 저녁 비행기로 서울로 떠난다.오늘은 그동안 가...
